# Causal Inference on Bayesian Networks for AI for Search and Rescue
## Author: Amanda Belden
### Master's Thesis


I followed guidance from the following on how to build Bayesian Networks in python:
Vidhi Chugh, "PGM 3: Python Implementation," WiCDS, Medium, https://medium.com/wicds/pgm-3-python-implementation-541603f70f5f




### Install Packages and Libraries

In [ ]:
#pip install pgmpy
#pip install feature_engine
#pip install graphviz
#pip install pydot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pgmpy.estimators import HillClimbSearch, MaximumLikelihoodEstimator, BicScore
from pgmpy.models import  BayesianNetwork
from pgmpy.inference import VariableElimination, CausalInference

from pgmpy.base import DAG

import networkx as nx

import pickle

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# BN - All Data

## Import SAR data

In [ ]:
df_sar = pd.read_csv("df_sar.csv")
df_sar.head()

Drop the non-binned columns.

In [ ]:
df_sar_categ = df_sar.copy()
df_sar_categ = df_sar_categ.drop(columns=["Unnamed: 0", "INCIDENT NUMBER", "LOCATION FOUND LAT", "LOCATION FOUND LONG", "LOCATION FOUND ELEVATION", 
                                    "MIN TEMP DEG F", "MAX TEMP DEG F", "MIN SNOW DEPTH IN", "MAX SNOW DEPTH IN", 
                                    "NUMBER OF RANGERS INVOLVED", 'LAST KNOWN POINT LAND CLASS', "ACTIVITY", "ACTIONS", "ROUTE FOLLOWED",
                                    "DETECTABILITY", "SITUATION", "CONTRIBUTING FACTOR",
                                    "TOTAL HOURS ISD", "TOTAL HOURS DON", 'Prop M Subjects', "REASON FOR BEING LOST",
                                    "Gender F Count", "Gender M Count", "Total Subjects", "Min Subject Age", "Max Subject Age",
                                    "EQUIPMENT USED", 'TECHNIQUE', 'start_time', 'TREATMENT PROVIDED BY RANGERS', 'TREATMENT PROVIDED BY OTHERS'])

In [ ]:
df_sar_categ['INCIDENT REGION'] = df_sar_categ['INCIDENT REGION'].astype(str)

Drop unnecessary columns

In [ ]:
df_sar_bn = df_sar_categ.drop(columns=['SUBJECT STATE', 'SUBJECT COUNTRY', 
                                       'closed_day_of_week', 'closed_month', 'notif_year', 'closed_year', 'notif_day_of_week', 
                                       'notif_month', 'closed_time', 'notif_time', 'NA EQUIPMENT IND', 'LOCATION FOUND LAT BINNED', 'MIN TEMP BINNED', 
                                       'LOCATION FOUND LONG BINNED', 'LOCATION FOUND ELEVATION BINNED',
                                       'MAX SNOW DEPTH BINNED', 'TOTAL HOURS DON BINNED', 'NA REASON IND', 'INCIDENT REGION ZONE', 
                                       'LAST KNOWN POINT COUNTY', 'INCIDENT COUNTY', 'Treatment Provided',
                                       'NA TECHNIQUE IND', 'Gender M Count Binned', 'Gender F Count Binned', 'Min Subject Age Binned',
                                       'LAST KNOWN POINT MAP NAME' ,'LAST KNOWN POINT STATE LAND NAME', 'LAST KNOWN POINT MUNICIPALITY',
                                       'NA ACTION IND', 'NA ROUTE IND', 'NA CONTR FACTOR IND'])

## Feature Selection

### Define Functions

#### Feature importance function

In [ ]:
def check_feature_importance(df, n, sample_size):
    all_variables = pd.DataFrame(list(df), columns=['Variables'])
    all_variables['Total BIC'] = 0

    for i in range(n):
        # Randomly select 10 columns from the DataFrame
        sampled_columns = np.random.choice(df.columns, size=sample_size, replace=False)

        # Create a new DataFrame containing only the sampled columns
        sampled_df = df[sampled_columns].copy()

        # train_X, test_X = train_test_split(sampled_df, test_size = 0.05, random_state = 1) 
        hc = HillClimbSearch(sampled_df, use_cache=True)
        best_model = hc.estimate(scoring_method=BicScore(sampled_df), max_iter=2000000, show_progress=False);
        edges = list(best_model.edges())
        model = BayesianNetwork(edges)

        bic = BicScore(sampled_df).score(model)
        selected_variables = list(model)

        # Filter the DataFrame based on the subset of variable names
        subset_df = all_variables[all_variables['Variables'].isin(selected_variables)]

        #Transform bic
        if bic == 0:
            bic_trans = 0
        elif bic < 0:
            bic_trans = 1 / abs(bic)
        else:
            bic_trans = bic

        # Add a number to the 'Total BIC' column for the subset of variables
        subset_df['Total BIC'] += bic_trans

        # Update the original DataFrame with the modified subset DataFrame
        all_variables.update(subset_df)

# Sort the DataFrame by 'Total BIC' column
    df_sorted = all_variables.sort_values(by='Total BIC')

    # Plot the DataFrame
    plt.figure(figsize=(10, 14))
    plt.barh(df_sorted['Variables'], df_sorted['Total BIC'], color='skyblue')
    plt.xlabel('Total of 1/ Abs(BIC)')
    plt.ylabel('Variables')
    plt.title('Feature Selection based on BIC Scores for Variables')
    plt.ylim(-0.5, len(df_sorted) - 0.5)
    # plt.xlim(-999999999, -99998999)
    # plt.grid(axis='x')
    plt.show();

#### Selection Function

In [ ]:
def check_feature_selection(df, n, sample_size):
    all_variables = pd.DataFrame(list(df), columns=['Variables'])
    all_variables['Total BIC'] = 0

    for i in range(n):
        # Randomly select 10 columns from the DataFrame
        sampled_columns = np.random.choice(df.columns, size=20, replace=False)

        # Create a new DataFrame containing only the sampled columns
        sampled_df = df[sampled_columns].copy()

        # train_X, test_X = train_test_split(sampled_df, test_size = 0.05, random_state = 1) 
        hc = HillClimbSearch(sampled_df, use_cache=True)
        best_model = hc.estimate(scoring_method=BicScore(sampled_df), max_iter=2000000, show_progress=False);
        edges = list(best_model.edges())
        model = BayesianNetwork(edges)

        bic = BicScore(sampled_df).score(model)
        selected_variables = list(model)

        # Filter the DataFrame based on the subset of variable names
        subset_df = all_variables[all_variables['Variables'].isin(selected_variables)]
        not_selected_variables = all_variables[~all_variables['Variables'].isin(selected_variables)]

        # Add a number to the 'Total BIC' column for the subset of variables
        subset_df['Total BIC'] += bic
        not_selected_variables['Total BIC'] += -99999

        # Update the original DataFrame with the modified subset DataFrame
        all_variables.update(subset_df)
        all_variables.update(not_selected_variables)
    # Sort the DataFrame by 'Total BIC' column
    df_sorted = all_variables.sort_values(by='Total BIC')

    # Plot the DataFrame
    plt.figure(figsize=(10, 14))
    plt.barh(df_sorted['Variables'], df_sorted['Total BIC'], color='skyblue')
    plt.xlabel('Total of BIC')
    plt.ylabel('Variables')
    plt.title('Feature Selection based on BIC Scores for Variables')
    plt.ylim(-0.5, len(df_sorted) - 0.5)
    # plt.xlim(-999999999, -99998999)
    # plt.grid(axis='x')
    plt.show();

#### Validation Function

In [ ]:
def check_full_cv(df, k=5):

    # Initialize an empty list to store evaluation scores
    models_and_scores = []

    # Iterate over the folds
    for i in range(k):
        # Perform Bayesian network structure learning
        hc = HillClimbSearch(df)
        best_model = hc.estimate(scoring_method=BicScore(df), max_iter=2000000, show_progress=False)
        
        # Create the Bayesian network model
        edges = list(best_model.edges())
        model = BayesianNetwork(edges)
        
        # Evaluate the model on the test set (you need to implement your own evaluation method)
        score = BicScore(df).score(model)
        # Store the model and its BIC score
        models_and_scores.append((model, score))

    score_list = [x[1] for x in models_and_scores]
    average_score = np.mean(score_list)
    print("Average BIC score:", average_score)
    return models_and_scores

# score_list = [x[1] for x in models_and_scores]
# print("List of scores: ", score_list)

# best_score = score_list[1]

# model_list = [x[0] for x in models_and_scores]
# best_model = model_list[1]
# # print("Best model edges:", best_model.edges())
# print("Best model BIC score:", best_score)

# # Compute the average score across all folds
# average_score = np.mean(score_list)
# print("Average score:", average_score)

### Model Building

Start with full dataset.

In [ ]:
# check_feature_selection(df_sar_bn, 150, 20)


Remove least important (bottom) 10 variables: 'start_day_of_week', 'start_year', 'Patrol Bicycle IND', 'Aircraft ACTIVITY IND', 'Chainsaw ACTIVITY IND', 'Evacuation by animal IND', 'Discarded equipment ACTION IND', 'Stranded ACTIVITY IND', 'Biking ACTIVITY IND', 'Flood Victim ACTIVITY IND'

Check new dataframe.

In [ ]:
df_sar_bn2 = df_sar_bn.copy()
df_sar_bn2 = df_sar_bn2.drop(columns=['start_day_of_week', 'start_year', 'Patrol Bicycle IND', 'Aircraft ACTIVITY IND', 'Chainsaw ACTIVITY IND', 
                                      'Evacuation by animal IND', 'Discarded equipment ACTION IND', 'Stranded ACTIVITY IND', 'Biking ACTIVITY IND', 
                                      'Flood Victim ACTIVITY IND'])

# check_feature_selection(df_sar_bn2, 150, 20)

In [ ]:
# models_and_scores2 = check_full_cv(df_sar_bn2)

Remove least important (bottom) five variables: 'Horseback riding ACTIVITY IND', 'Natural clearing ROUTE IND', 'Weather', 'Visual Sighting from Air DETECT IND', 'TOTAL HOURS ISD BINNED'
Check new dataframe.

In [ ]:
df_sar_bn3 = df_sar_bn2.copy()
df_sar_bn3 = df_sar_bn3.drop(columns=['Horseback riding ACTIVITY IND', 'Natural clearing ROUTE IND',
                                      'Visual Sighting from Air DETECT IND', 'TOTAL HOURS ISD BINNED'])

# check_feature_selection(df_sar_bn3, 150, 20)

In [ ]:
# models_and_scores3 = check_full_cv(df_sar_bn3)

Remove least important (bottom) five variables: 'Natural clearing ROUTE IND', 'Horseback riding ACTIVITY IND', 'Visual Sighting from Air DETECT IND', 
                                      'Camping ACTIVITY IND', 'Weather'

Check new dataframe.

In [ ]:
df_sar_bn4 = df_sar_bn3.copy()
df_sar_bn4 = df_sar_bn4.drop(columns=['Toward landmark ROUTE IND', 'Toward civilization ROUTE IND', 'Max Subject Age Binned', 
                                      'Camping ACTIVITY IND', 'Ice rescue gear IND'])

# check_feature_selection(df_sar_bn4, 150, 20)

In [ ]:
# models_and_scores4 = check_full_cv(df_sar_bn4)

In [ ]:
# score_list = [x[1] for x in models_and_scores4]
# print(score_list)

Model not changing much. Move to pruning.

In [ ]:
hc = HillClimbSearch(df_sar_bn4, use_cache=True)
best_model = hc.estimate(scoring_method=BicScore(df_sar_bn4), max_iter=2000000, show_progress=False)
edges = list(best_model.edges())
model = BayesianNetwork(edges)

print(edges)

In [ ]:
BicScore(df_sar_bn4).score(model)

In [ ]:
%matplotlib inline

# nx.draw(model, with_labels=True)
plt.figure(figsize=(40, 45))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=2, font_size=11,
        arrows=True, node_color='skyblue', edge_color='silver', font_color='black', connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

## Model Pruning

In [ ]:
df_sar_bn5 = df_sar_bn4.copy()
df_sar_bn5 = df_sar_bn5.drop(columns=['Offroad/Motor Vehicle/ATV ACTIVITY IND', 'INCIDENT REGION', 'Equipment failure CONTR FACTOR IND',
                                      'start_month', 'Runaway ACTIVITY IND'])

# check_feature_importance(df_sar_bn6, 100, 30)
# models_and_scores5 = check_full_cv(df_sar_bn5, 3)

In [ ]:
hc = HillClimbSearch(df_sar_bn5, use_cache=True)
best_model = hc.estimate(scoring_method=BicScore(df_sar_bn5), max_iter=2000000, show_progress=False)
edges = list(best_model.edges())
model = BayesianNetwork(edges)

print(edges)
%matplotlib inline

# nx.draw(model, with_labels=True)
plt.figure(figsize=(40, 45))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=2, font_size=11,
        arrows=True, node_color='skyblue', edge_color='silver', font_color='black', connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

In [ ]:
BicScore(df_sar_bn5).score(model)

In [ ]:
df_sar_bn6 = df_sar_bn5.copy()
df_sar_bn6 = df_sar_bn6.drop(columns=['Snowmobile IND', 'Dive Team IND', 'LEAD ORGANIZATION NAME'])

# check_feature_importance(df_sar_bn6, 100, 30)
# models_and_scores7 = check_full_cv(df_sar_bn6)

In [ ]:
hc = HillClimbSearch(df_sar_bn6, use_cache=True)
best_model = hc.estimate(scoring_method=BicScore(df_sar_bn6), max_iter=2000000, show_progress=False)
edges = list(best_model.edges())
model = BayesianNetwork(edges)

print(edges)
%matplotlib inline

# nx.draw(model, with_labels=True)
plt.figure(figsize=(40, 45))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=2, font_size=11,
        arrows=True, node_color='skyblue', edge_color='silver', font_color='black', connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

In [ ]:
BicScore(df_sar_bn6).score(model)

In [ ]:
df_sar_bn7 = df_sar_bn6.copy()
df_sar_bn7 = df_sar_bn7.drop(columns=['Skiing ACTIVITY IND', 'Action IND', 'Survival Action IND'])

# check_feature_importance(df_sar_bn7, 100, 30)
# models_and_scores7 = check_full_cv(df_sar_bn7)

In [ ]:
hc = HillClimbSearch(df_sar_bn7, use_cache=True)
best_model = hc.estimate(scoring_method=BicScore(df_sar_bn7), max_iter=2000000, show_progress=False)
edges = list(best_model.edges())
model = BayesianNetwork(edges)

print(edges)
%matplotlib inline

# nx.draw(model, with_labels=True)
plt.figure(figsize=(40, 45))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=2, font_size=11,
        arrows=True, node_color='skyblue', edge_color='silver', font_color='black', connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

In [ ]:
BicScore(df_sar_bn7).score(model)

In [ ]:
df_sar_bn8 = df_sar_bn7.copy()
df_sar_bn8 = df_sar_bn8.drop(columns=['Natural drainage ROUTE IND', 'Weather CONTR FACTOR IND', 'MAX TEMP BINNED', 'Cross country ROUTE IND', 
                                      'Did not travel ROUTE IND'])

# check_feature_importance(df_sar
hc = HillClimbSearch(df_sar_bn8, use_cache=True)
best_model = hc.estimate(scoring_method=BicScore(df_sar_bn8), max_iter=2000000, show_progress=False)
edges = list(best_model.edges())
model = BayesianNetwork(edges)

print(edges)
%matplotlib inline

# nx.draw(model, with_labels=True)
plt.figure(figsize=(40, 40))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=2, font_size=11,
        arrows=True, node_color='turquoise', edge_color='silver', font_color='black', connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

In [ ]:
BicScore(df_sar_bn8).score(model)

In [ ]:
# check_feature_selection(df_sar_bn8, 100, 25)

In [ ]:
df_sar_bn9 = df_sar_bn8.copy()
df_sar_bn9 = df_sar_bn9.drop(columns=['Climbing:Rock/Ice ACTIVITY IND', 'Mobile DETECT IND'])

# check_feature_importance(df_sar
hc = HillClimbSearch(df_sar_bn9, use_cache=True)
best_model = hc.estimate(scoring_method=BicScore(df_sar_bn9), max_iter=2000000, show_progress=False)
edges = list(best_model.edges())
model = BayesianNetwork(edges)

print(edges)
%matplotlib inline

# nx.draw(model, with_labels=True)
plt.figure(figsize=(40, 40))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=2, font_size=11,
        arrows=True, node_color='skyblue', edge_color='silver', font_color='black', connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

In [ ]:
plt.figure(figsize=(23,20))
pos = nx.circular_layout(model)
nx.draw(model, pos=pos, with_labels=True, node_size=3000, width=1, font_size=11,
        arrows=True, node_color='skyblue', edge_color='black', font_color=(0.07,0.07,0.07), connectionstyle='arc3,rad=0.2')
plt.savefig('full_bn.png', bbox_inches='tight')
plt.show();

In [ ]:
BicScore(df_sar_bn9).score(model)

# Inference

In [ ]:
# Fitting the data to the model using Maximum Likelihood Estimator
model.fit(df_sar_bn9, estimator=MaximumLikelihoodEstimator)

# Doing exact inference using Variable Elimination
infer = VariableElimination(model)

Conditional Dist.

In [ ]:
for i in ['No Medical Assistance Required', 'Required Medical Treatment', 'Deceased']:
    print("CONDITION WHEN FOUND:", i)
    print(infer.query(variables=['FOUND IN SEARCH AREA'], evidence = {'CONDITION WHEN FOUND': i}))

In [ ]:
from pgmpy.base import DAG
G = DAG()
G.add_nodes_from(nodes=['CONDITION WHEN FOUND', 'FOUND IN SEARCH AREA'])
G.add_edge(u='CONDITION WHEN FOUND', v='FOUND IN SEARCH AREA')
plt.figure(figsize=(12,10))
pos = nx.circular_layout(G)
nx.draw(G, pos=pos, with_labels=True, node_size=2000, width=1, font_size=11,
        arrows=True, node_color='skyblue', edge_color='black', font_color=(0.07,0.07,0.07), connectionstyle='arc3,rad=0.2')
plt.show();

In [ ]:
for i in ['0-4am', '4-8am', '8am-12pm', '12-4pm', '4-8pm', '8pm-12am']:
    print("start_time_interval:", i)
    print(infer.query(variables=['Injured SITUATION IND'], evidence = {'start_time_interval': i}))

In [ ]:
G = DAG()
G.add_nodes_from(nodes=['Injured SITUATION IND', 'start_time_interval'])
G.add_edge(u='Injured SITUATION IND', v='start_time_interval')
G.add_edges_from(ebunch=[('Illness SITUATION IND', 'Injured SITUATION IND'), ('RESPONSE TYPE', 'Injured SITUATION IND'), ('Stranded SITUATION IND', 'Injured SITUATION IND'), ('Injured SITUATION IND', 'Darkness CONTR FACTOR IND')])
plt.figure(figsize=(10,6))
pos = nx.circular_layout(G)
nx.draw(G, pos=pos, with_labels=True, node_size=2000, width=1, font_size=11,
        arrows=True, node_color='skyblue', edge_color='black', font_color=(0.07,0.07,0.07), connectionstyle='arc3,rad=0.2')
plt.show();

In [ ]:
for i in ['Yes', 'No']:
    print("Injured SITUATION IND:", i)
    print(infer.query(variables=['start_time_interval'], evidence = {'Injured SITUATION IND': i}))

In [ ]:
for i in ['0-4am', '4-8am', '8am-12pm', '12-4pm', '4-8pm', '8pm-12am']:
    print("start_time_interval:", i)
    print(infer.query(variables=['RESPONSE TYPE'], evidence = {'start_time_interval': i}))

In [ ]:
for i in ['Yes', 'No']:
    print("Darkness CONTR FACTOR IND:", i)
    print(infer.query(variables=['Injured SITUATION IND'], evidence = {'Darkness CONTR FACTOR IND': i}))

In [ ]:
for i in ['Recovery', 'Rescue', 'Search']:
    print("RESPONSE TYPE:", i)
    print(infer.query(variables=['Injured SITUATION IND'], evidence = {'RESPONSE TYPE': i}))

In [ ]:
for i in ['0% M', '1-49% M', '50% M', '51-99% M', '100% M']:
    print("Prop M Subjects Binned:", i)
    print(infer.query(variables=['Assist/own power IND'], evidence = {'Prop M Subjects Binned': i}))

In [ ]:
G = DAG()
G.add_edges_from(ebunch=[('Missing SITUATION IND', 'Dementia IND'), ('Land Class', 'Missing SITUATION IND'), ('CONDITION WHEN FOUND', 'Missing SITUATION IND'), ('Missing SITUATION IND', 'Aircraft/Helicopter/UAV IND'), ('Aircraft/Helicopter/UAV IND', 'NUMBER RANGERS BINNED')])
plt.figure(figsize=(14,6))
pos = nx.circular_layout(G)
nx.draw(G, pos=pos, with_labels=True, node_size=4000, width=1, font_size=12,
        arrows=True, node_color='skyblue', edge_color='black', font_color=(0.07,0.07,0.07), connectionstyle='arc3,rad=0.2')
plt.show();

In [ ]:
for i in ['Yes', 'No']:
    print("Missing SITUATION IND:", i)
    print(infer.query(variables=['Land Class'], evidence = {'Missing SITUATION IND': i}))

In [ ]:
for i in ['Yes', 'No']:
    print("Dementia IND:", i)
    print(infer.query(variables=['Missing SITUATION IND'], evidence = {'Dementia IND': i}))

In [ ]:
for i in ['Yes', 'No']:
    print("Missing SITUATION IND':", i)
    print(infer.query(variables=['Aircraft/Helicopter/UAV IND'], evidence = {'Missing SITUATION IND': i}))

Counterfactual

In [ ]:
print(infer.query(variables=['Injured SITUATION IND'], evidence = {'Darkness CONTR FACTOR IND': 'Yes'}))

In [ ]:
print(infer.query(variables=['Injured SITUATION IND'], evidence = {'Darkness CONTR FACTOR IND': 'No'}))

In [ ]:
# from pgmpy.inference import CausalInference
causal_infer = CausalInference(model)
# causal_infer.query(['Injured SITUATION IND'], do={'Terrain CONTR FACTOR IND': 'No'}, evidence={'Darkness CONTR FACTOR IND': 'Yes'})

In [ ]:
print(causal_infer.query(['Total Subjects Binned'], do={'Injured SITUATION IND': 'Yes'}, evidence={'Prop M Subjects Binned': '100% M'}))
print(causal_infer.query(['Total Subjects Binned'], do={'Injured SITUATION IND': 'No'}, evidence={'Prop M Subjects Binned': '100% M'}))

In [ ]:
print(causal_infer.query(['CONDITION WHEN FOUND'], do={'Missing SITUATION IND': 'Yes'}))
print(causal_infer.query(['CONDITION WHEN FOUND'], do={'Missing SITUATION IND': 'No'}))

In [ ]:
print(causal_infer.query(['NUMBER RANGERS BINNED'], do={'Missing SITUATION IND': 'Yes'}))
print(causal_infer.query(['NUMBER RANGERS BINNED'], do={'Missing SITUATION IND': 'No'}))

In [ ]:
print(causal_infer.query(['Darkness CONTR FACTOR IND'], do={'Stranded SITUATION IND': 'Yes'}))
print(causal_infer.query(['Darkness CONTR FACTOR IND'], do={'Stranded SITUATION IND': 'No'}))

In [ ]:
print(causal_infer.query(['Prop M Subjects Binned'], do={'Assist/own power IND': 'Yes'}, evidence={'Missing SITUATION IND': 'Yes'}))
print(causal_infer.query(['Prop M Subjects Binned'], do={'Assist/own power IND': 'No'}, evidence={'Missing SITUATION IND': 'Yes'}))

In [ ]:
print(causal_infer.query(['Prop M Subjects Binned'], do={'Injured SITUATION IND': 'Yes'}))
print(causal_infer.query(['Prop M Subjects Binned'], do={'Injured SITUATION IND': 'No'}))

# Pickle Model

In [ ]:
bn_full_pickle = open('bn_full.pickle', 'wb') 

# Write DT model to the file
pickle.dump(model, bn_full_pickle) 

# Close the file
bn_full_pickle.close() 

In [ ]:
df_sar_bn9['Land Class'].value_counts()